In [ ]:
import sklearn
import pandas as pd
import numpy as np
import nltk
import sklearn
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import matplotlib
from sklearn.model_selection import cross_val_score
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn.model_selection import train_test_split
import re
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
import pickle
from wordcloud import WordCloud

In [ ]:
full_reviewed_data_drop = pd.read_csv('../../input_data/full_datasets/fully_hand_labelled_texts.csv')
indices = pd.read_csv('medrobertanl-haga_smoking_predictions.csv', delimiter=';')

In [ ]:
test_set = full_reviewed_data_drop.loc[indices['index']]

In [ ]:
test_set_check = test_set[['text', 'Roken']]

In [ ]:
test_set.text = test_set.text.str.replace("_x000D_\n", "\t")


In [ ]:
test_set_check['Roken'] = test_set_check['Roken'].str.replace('Geen gebruiker', 'Voormalige gebruiker')

In [ ]:
test_set_check = test_set_check[(test_set_check.Roken == 'Huidige gebruiker') | (test_set_check.Roken == 'Voormalige gebruiker')]

In [ ]:
test_set_check

In [ ]:
def apply_string_matching(row):
    text = row['text'].lower()
    if "roken+" in text or "roken +" in text:
        return 'Huidige gebruiker'
    if 'door' in text and 'stoppen' in text:
        return 'Voormalige gebruiker'
    if 'gevolg' in text and 'stoppen' in text:
        return 'Voormalige gebruiker'
    if ('gestopt' in text or 'gestaakt' in text) and not ('niet' in text or 'was' in text):
        return 'Voormalige gebruiker'
    if 'tot' in text and 'jaar' in text:
        return 'Voormalige gebruiker'
    if 'roken-' in text or 'roken -' in text:
        return 'Voormalige gebruiker'
    if 'stoppen' in text or 'staken' in text or 'verminderen' in text or 'persisterend' in text:
        return 'Huidige gebruiker'
    if 'per dag' in text or 'packyears' in text or "/dag" in text or "pakje" in text or "pack" in text:
        return "Huidige gebruiker"
    if 'pd' in text or "py" in text:
        return 'Huidige gebruiker'

test_set_check['Prediction'] = test_set_check.apply(lambda row: apply_string_matching(row), axis=1)

In [ ]:
test_set_check_no_na = test_set_check.dropna()
test_set_check_no_na

In [ ]:
print(np.mean(test_set_check_no_na['Prediction'] == test_set_check_no_na['Roken']))
cm = confusion_matrix(test_set_check_no_na['Roken'], test_set_check_no_na['Prediction'])
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

plt.show()
print(metrics.classification_report(test_set_check_no_na['Roken'], test_set_check_no_na['Prediction']))